# UNIT Project in Colab
[on GitHub](https://github.com/mingyuliutw/UNIT)


## Workspace Setup
Bash commands setting up the current session for Juputer Notebook

### Google Drive Folder Mounting
Following code mounts your google.drive to container. Choose **one** of next **two** methods.


#### Mounting (built-in)

#### Mounting With Fuse Driver (google-drive-ocamlfuse)

In [0]:
# Mount Google.Drive with google-drive-ocamlfuse
% cd /content/
print('Installing required software')
! apt-get install -y -qq software-properties-common module-init-tools 2>&1 > /dev/null
print('Add apt-repository with Google.Drive Fuse')
! add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
print('Updating packages...')
! apt-get update -y -qq
print()'Installing google-drive-ocamlfuse fuse...')
! apt-get install -y -qq google-drive-ocamlfuse fuse
print('Authenticate Fuse in Google.Drive...')
from google.colab import auth
from oauth2client.client import GoogleCredentials
import getpass
auth.authenticate_user()
creds = GoogleCredentials.get_application_default()
! google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass('Enter auth code here: ')
! echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
print('Authenticated!')
print('Creating mount directory')
! mkdir ./drive2
print('Mounting...')
! google-drive-ocamlfuse ./drive2
! ls ./drive2
print('Mounted!')

## SSH Tunnel

In [0]:
print('Generate root password')
import secrets, string
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for i in range(20))

print('Download ngrok')
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
print('Setup sshd')
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
print('Set root password')
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

print('Run sshd')
get_ipython().system_raw('/usr/sbin/sshd -D &')

print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

print('Create tunnel')
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
print('Get public address')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Project

### Install Dependencies
One time installation

In [0]:
!conda install pytorch -c pytorch
!pip3 install torchvision

In [0]:
print('Cloning GitHub project...')
# !git clone https://github.com/mingyuliutw/UNIT.git
# print('Installing system packages...')
# !apt-get install -y -qq axel imagemagick 2>&1 > /dev/null
print('Installing project dependencies...')
! pip3 install http://download.pytorch.org/whl/cu90/torch-0.4.1-cp36-cp36m-win_amd64.whl
! pip3 install torchvision
!pip3 install tensorboard tensorboardX 2>&1 > /dev/null
# !mkdir /content/UNIT/models
# !cp -r /content/drive/UNIT/models/ /content/UNIT/
print('Complete!')

## Train

In [0]:
% cd '/content/drive/UNIT'
# print('Training a model for sketches of handbags to images of handbags translation')
# ! bash scripts/unit_demo_train_edges2handbags.sh
# print('Training a model for sketches of shoes to images of shoes translation')
# ! bash scripts/unit_demo_train_edges2shoes.sh
print('Training a model for Yosemite summer 256x256 images to Yosemite winter 256x256 image translation')
! bash scripts/unit_demo_train_summer2winter_yosemite256.sh
print('Fully Trained!')

## Test

In [0]:
from IPython.display import Image, display

% cd '/content/drive/UNIT'
print('Testing: gta2city')
! python test.py --trainer UNIT --config configs/unit_gta2city_list.yaml --input inputs/gta3.jpg --output_folder results/gta2city --checkpoint models/unit_gta2city.pt --a2b 1
# print('Testing: city2gta')
# ! python test.py --trainer UNIT --config configs/unit_gta2city_list.yaml --input inputs/city_example.jpg --output_folder results/city2gta --checkpoint models/unit_gta2city.pt --a2b 0
# print('Input')
# display(Image('/content/drive/UNIT/results/city2gta/input.jpg', width=800))
# print('Output')
# display(Image('/content/drive/UNIT/results/city2gta/output.jpg', width=800))
print('Input')
display(Image('/content/drive/UNIT/results/gta2city/input.jpg', width=800))
print('Output')
display(Image('/content/drive/UNIT/results/gta2city/output.jpg', width=800))
